In [21]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.layers import Dense, Dropout, Flatten
import argparse 

import os
import tarfile

In [10]:
img_size=224
num_classes = 6
epochs = 5 
batch_size = 16
train_data_dir = 'data_train'

In [5]:
train=pd.read_csv('train-scene/train.csv')
test=pd.read_csv('test.csv')
image_path='train-scene/train/'

In [6]:
test_img=[]
for i in range(len(test)):
    temp_img=image.load_img(image_path+test['image_name'][i],target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    test_img.append(temp_img)

test_img=np.array(test_img)
test_img=preprocess_input(test_img)

In [30]:
train_datagen = image.ImageDataGenerator(
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2)

c:\users\rohit\appdata\local\programs\python\python36\lib\site-packages\keras_preprocessing\image\image_data_generator.py:346: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [31]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation') # set as validation data



Found 13630 images belonging to 6 classes.
Found 3404 images belonging to 6 classes.


In [32]:
from keras.optimizers import SGD

In [33]:
model = Sequential()
model.add(ResNet50(include_top=False,pooling='avg',weights ='imagenet'))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(6,activation='softmax'))


model.layers[0].trainable = False
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


In [29]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/5
851/851 [==============================] - 179s 210ms/step - loss: 0.7076 - acc: 0.7438 - val_loss: 0.6904 - val_acc: 0.7733
Epoch 2/5
851/851 [==============================] - 168s 198ms/step - loss: 0.5385 - acc: 0.8134 - val_loss: 0.6736 - val_acc: 0.7736
Epoch 3/5
851/851 [==============================] - 171s 200ms/step - loss: 0.4861 - acc: 0.8337 - val_loss: 0.6310 - val_acc: 0.7456
Epoch 4/5
851/851 [==============================] - 170s 199ms/step - loss: 0.4859 - acc: 0.8307 - val_loss: 0.7139 - val_acc: 0.7488
Epoch 5/5
851/851 [==============================] - 168s 197ms/step - loss: 0.4684 - acc: 0.8413 - val_loss: 0.7543 - val_acc: 0.7347
